In [ ]:
import numpy
import pandas
import re
from pandas import DataFrame
from sklearn.linear_model import SGDClassifier

In [ ]:
train = pandas.read_csv("linear_train.txt", header=None)

In [ ]:
train

In [ ]:
alphabet = "абвгдеёжзийклмнопрстуфхцчшщъыьэюя’'-"
words = numpy.array([
    line.lower()
    for line in numpy.array(train)[:,0]
])
targets = numpy.array([
    target for target in numpy.array(train)[:,1]
])

In [ ]:
def consists_of_alphabet(word):
    for character in word:
        if character not in alphabet:
            return False
    return True

In [ ]:
good_words = [
    word for word in words if consists_of_alphabet(word)
]

good_targets = [
    answer for word, answer in zip(words, targets) if consists_of_alphabet(word)
]

strange_lines = {
    (word, answer) for word, answer in zip(words, targets) if not consists_of_alphabet(word)
}

targeted_strange_lines = {
    (word, answer) for word, answer in strange_lines if answer == 1
}

In [ ]:
len(good_targets), len(good_words)

In [ ]:
endings_length = 5
endings = {
    word[-i-1:]
    for i in range(endings_length)
    for word in good_words
}

In [ ]:
def ends_with(word, ending):
    match = re.match(r".*" + ending, word)
    return (match is not None) and (match.end() == len(word))

In [ ]:
regulars = [re.compile(r".*" + ending) for ending in endings]

In [ ]:
def get_feature(i, j, words=good_words, regulars=regulars):
    match = regulars[j].match(words[i])
    return 1 if ((match is not None) and (match.end() == len(words[i]))) else 0

In [ ]:
rows = []
columns = []
features = []
for i in range(len(good_words)):
    for j in range(len(regulars)):
        feature = get_feature(i, j)
        if feature == 1:
            rows.append(i)
            columns.append(j)
            features.append(1)

In [ ]:
features = numpy.array([
    [1 if ends_with(word, ending) else 0 for ending in endings]
    for word in good_words
])